In [1]:
import sys
import boto3
import json
print(sys.executable)

/Users/jadyliu/rag-demo/.venv/bin/python


In [2]:
client = boto3.client('bedrock-agent-runtime', region_name='us-west-2')
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name='us-west-2')
kb_id = "UQUAYYWKQQ"
summary_kb_id = "V0VELRBYVL"
model_id_1 = "arn:aws:bedrock:us-west-2::foundation-model/mistral.mistral-7b-instruct-v0:2"
model_id = "arn:aws:bedrock:us-west-2::foundation-model/mistral.mistral-large-2402-v1:0"

## RAG demo
### Virtual assistant Q&A from Berkshire Hathaway Stakeholder letters

In [3]:
def test_kb(kb_id: str, query: str):
    request = {
        "input": {
            "text": query
        },
        "retrieveAndGenerateConfiguration": {
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                "knowledgeBaseId": kb_id,
                "modelArn": model_id
            }
        }
    }

    try:
        response = client.retrieve_and_generate(**request)
        
        # Print the main generated response
        print("\n=== Generated Response ===")
        print(response['output']['text'])
        
        # Print citations and their sources
        print("\n=== Citations and Sources ===")
        for citation in response['citations']:
            for ref in citation['retrievedReferences']:
                # Get metadata directly from the metadata field
                metadata = ref['metadata']
                
                print("\nSource Document:")
                print(f"File: {metadata.get('filename')}")
                print(f"Title: {metadata.get('title')}")
                print(f"Page Number: {metadata.get('x-amz-bedrock-kb-document-page-number')}")
                print("\nCitation Text:")
                print(citation['generatedResponsePart']['textResponsePart']['text'])
            
        # Print session ID for reference
        print(f"\nSession ID: {response['sessionId']}")
        
    except Exception as e:
        print(f"Error: {str(e)}")


# Example usage:
query = "What are the key points discussed in the letters of 2023?"
test_kb(kb_id, query)



=== Generated Response ===
The letters of 2023 discuss several key points. Firstly, they mention the use of the words "mistake" or "error" 16 times in the letters to shareholders, highlighting the transparency of the company. Secondly, they discuss the importance of honesty and avoiding "happy talk and pictures" in reporting to shareholders. Thirdly, they mention the future replacement of the CEO by Greg Abel. Lastly, they discuss the story of Pete Liegl and his contribution to the company. The letters also discuss the operating results, both factual and perceived. They highlight the importance of understanding accounting terms and the power of incentives. They also mention the "bottom line" net earnings for the years 2021, 2022, and 2023.

=== Citations and Sources ===

Source Document:
File: 2024-letters.pdf
Title: To the Shareholders of Berkshire Hathaway Inc.
Page Number: 1.0

Citation Text:
The letters of 2023 discuss several key points. Firstly, they mention the use of the words

In [4]:
def test_kb_with_filter(kb_id: str, query: str):
    
    request = {
        "input": {
            "text": query
        },
        "retrieveAndGenerateConfiguration": {
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                "knowledgeBaseId": kb_id,
                "modelArn": model_id,
                "retrievalConfiguration": {
                    "vectorSearchConfiguration": {
                        "numberOfResults": 3,
                        "filter": {
                            "equals": {
                                "key": "filename",
                                "value": "2023-letters.pdf"
                            }
                        }
                    }
                }
            }
        }
    }

    try:
        response = client.retrieve_and_generate(**request)
        
        # Print the main generated response
        print("\n=== Generated Response ===")
        print(response['output']['text'])
        
        # Print citations and their sources
        print("\n=== Citations and Sources ===")
        for citation in response['citations']:
            for ref in citation['retrievedReferences']:
                # Get metadata directly from the metadata field
                metadata = ref['metadata']
                
                print("\nSource Document:")
                print(f"File: {metadata.get('filename')}")
                print(f"Title: {metadata.get('title')}")
                print(f"Page Number: {metadata.get('x-amz-bedrock-kb-document-page-number')}")
                print("\nCitation Text:")
                print(citation['generatedResponsePart']['textResponsePart']['text'])
            
        # Print session ID for reference
        print(f"\nSession ID: {response['sessionId']}")
        
    except Exception as e:
        print(f"Error: {str(e)}")


# Example usage:
query = "What are the key points discussed in the letters of 2023?"
test_kb_with_filter(kb_id, query)



=== Generated Response ===
The key points discussed in the letters of 2023 include the presentation of operating earnings, the outlook for the full year, and the performance of specific business segments. The operating earnings for 2023 were $37,350 million, compared to $30,853 million in 2022. The outlook for 2023 suggested that most non-insurance businesses would face lower earnings, but this decline would be cushioned by decent results at BNSF and Berkshire Hathaway Energy (BHE). However, the expectations for both BNSF and BHE were not met. Insurance performed as expected. Another key point discussed is the difference between "net earnings (loss)" and "operating earnings". The net earnings figures for 2021, 2022, and 2023 were $90 billion, ($23 billion), and $96 billion, respectively. However, Berkshire prefers to report "operating earnings", which exclude unrealized capital gains or losses. These figures were $27.6 billion for 2021, $30.9 billion for 2022, and $37.4 billion for 20

In [5]:
def test_kb_complex_filter(kb_id: str):    
    questions = [
        {
            "name": "Recent Insurance Performance",
            "query": "How has Berkshire's insurance business performed in recent years, particularly focusing on underwriting profits and investment income after 2022?",
            "filter": {
                "orAll": [
                    {
                        "equals": {
                            "key": "filename",
                            "value": "2023-letters.pdf"
                        }
                    },
                    {
                        "equals": {
                            "key": "filename",
                            "value": "2024-letters.pdf"
                        }
                    }
                ]
            }
        },
        {
            "name": "Charlie Munger Tribute",
            "query": "Find specific mentions of Charlie Munger as the architect of Berkshire Hathaway.",
            "filter": {
                "andAll": [
                    {
                        "equals": {
                            "key": "filename",
                            "value": "2023-letters.pdf"
                        }
                    },
                    {
                        "stringContains": {
                            "key": "title",
                            "value": "Architect"
                        }
                    }
                ]
            }
        },
       {
            "name": "Operating Performance Comparison",
            "query": "Compare Berkshire's operating earnings and business unit performance between 2022 and 2023. Focus on BNSF and Energy businesses.",
            "filter": {
                "in": {
                    "key": "filename",
                    "value": ["2022-letters.pdf", "2023-letters.pdf"]
                }
            }
        }
    ]
    
    for question in questions:
        print(f"\n=== {question['name']} ===")
        print(f"Question: {question['query']}")
        
        request = {
            "input": {
                "text": question['query']
            },
            "retrieveAndGenerateConfiguration": {
                "type": "KNOWLEDGE_BASE",
                "knowledgeBaseConfiguration": {
                    "knowledgeBaseId": kb_id,
                    "modelArn": model_id,
                    "retrievalConfiguration": {
                        "vectorSearchConfiguration": {
                            "numberOfResults": 4,
                            "filter": question['filter']
                        }
                    }
                }
            }
        }
        
        try:
            response = client.retrieve_and_generate(**request)
            print("\nAnswer:")
            print(response['output']['text'])
            
            # print("\nSources:")
            # for citation in response['citations']:
            #     for ref in citation['retrievedReferences']:
            #         metadata = ref['metadata']
            #         print(f"\nFrom {metadata['filename']} (Page {metadata.get('x-amz-bedrock-kb-document-page-number', 'N/A')})")
                    
        except Exception as e:
            print(f"Error: {str(e)}")
        
        print("\n" + "="*50)

# Example usage:
test_kb_complex_filter(kb_id)



=== Recent Insurance Performance ===
Question: How has Berkshire's insurance business performed in recent years, particularly focusing on underwriting profits and investment income after 2022?

Answer:
Berkshire's insurance business has seen a significant increase in earnings, led by the performance of GEICO. In 2024, the insurance business delivered a major increase in earnings, with GEICO being a long-held gem that underwent major repolishing. In terms of underwriting profits, over the past two decades, Berkshire's insurance business has generated $32 billion of after-tax profits from underwriting, about 3.3 cents per dollar of sales after income tax. The float has grown from $46 billion to $171 billion during this period. In 2023, the insurance-underwriting profit was $5,428 million, a significant improvement from a loss of $30 million in 2022. For investment income, Berkshire saw a rise from $6,484 million in 2022 to $9,567 million in 2023.


=== Charlie Munger Tribute ===
Questio

# Function tooling with Chain of thought:

1. **get_filename_summary:**
- Performs a vector search query against the summaries Knowledge Base containing document summaries in JSON
- Takes a text query as input
- Retrieves up to 1 most relevant results based on semantic similarity

2. **construct_metadata_filter:**
- Creates a metadata filter structure for filename-based filtering
- Generates an \"equals\" filter condition when a valid detail filename is present

3. **process_query:**
- Executes a filtered search against the documents Knowledge Base
- Takes both query text and a filename as parameters
- Creates the metadata filter using the *construct_metadata_filter*
- Applies metadata filtering to limit results to specific files using the *filename* metadata
- Returns up to 2 most relevant results that match both the semantic query and filename filter


In [6]:
def get_filename_summary(text):
    response = client.retrieve(
        knowledgeBaseId=summary_kb_id,
        retrievalConfiguration={
            "vectorSearchConfiguration": {
                "numberOfResults": 2
            }
        },
        retrievalQuery={
            'text': text
        }
    )
    return response

def process_query(text, filename):
    metadata_filter = construct_metadata_filter(filename)
    print('Here is the prepared metadata filters:')
    print(metadata_filter)

    response = client.retrieve(
        knowledgeBaseId=kb_id,
        retrievalConfiguration={
            "vectorSearchConfiguration": {
                "filter": metadata_filter,
                "numberOfResults": 2
            }
        },
        retrievalQuery={
            'text': text
        }
    )
    return response

def construct_metadata_filter(filename):
    if not filename:
        return None
    metadata_filter = {"equals": []}

    if filename and filename != 'unknown':
        metadata_filter = {
            "equals": {
                "key": "filename",
                "value": filename
            }
        }

    return metadata_filter if metadata_filter["equals"] else None


def process_tool_call(tool_name, tool_input):
    if tool_name == "get_filename_summary":
        return get_filename_summary(tool_input["query"])
    elif tool_name == "process_query":
        return process_query(tool_input["query"], tool_input["filename"])


### Mistral AI function tooling definition 
```
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_filename_summary",
            "description": "Useful to retrieve the filename of the document associated to the user's query. Returns the filename, title and summary",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The decomposed query questions"
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "process_query",
            "description": "Retrieves specific information related to the user's query using a filter containing the name of the filename in order to get specific details from the relevant document. Returns a set of relevant chunks",
            "parameters": {
                "type": "object",
                "properties": {
                    "filename": {
                        "type": "string",
                        "description": "The name of the filename corresponding to the relevant document"
                    },
                    "query": {
                        "type": "string",
                        "description": "The decomposed query"
                    }                    },
                "required": ["filename", "query"]
            },
        },
    }
]
```

In [7]:
tools = [
    {
        "toolSpec": {
            "name": "get_filename_summary",
            "description": "Useful to retrieve the filename of the document associated to the user's query. Returns the filename, title and summary",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The decomposed query"
                        }
                    },
                    "required": ["query"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "process_query",
            "description": "Retrieves specific information related to the user's query using a filter containing the name of the filename in order to get specific details from the relevant document. Returns a set of relevant chunks",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "filename": {
                            "type": "string",
                            "description": "The name of the filename corresponding to the relevant document"
                        },
                        "query": {
                            "type": "string",
                            "description": "The decomposed query"
                        }                    },
                    "required": ["filename", "query"]
                }
            }
        }
    }
]

In [8]:
system_message = """
You are an advanced AI assistant. Your task is to break down complex queries and utilize appropriate tools to provide accurate and comprehensive responses.

# For each user {query}:
Follow these steps:
## STEP 1: QUERY ANALYSIS AND DECOMPOSITION
Extract and identify the key concepts, terms, and intent from the user's question to form effective search parameters.
## STEP 2: METADATA SEARCH WITH TOOLS
Use get_filename_summary() to identify the most relevant document(s) name based on the analyzed query terms. After executing the get_filename_summary tool, you should extract the filename ONLY from the "filename" in the "metadataAttributes" dictionary.
## STEP 3: FILTERED INFORMATION RETRIEVAL
Use process_query() to search within the identified document(s) to extract specific relevant passages.
## STEP 4: RESPONSE FORMULATION
Synthesize the retrieved information into a clear, direct answer that addresses the original query with appropriate source references.

# Available Tools:
1. get_filename_summary(text: str)
- Searches the summary knowledge base to identify relevant document names
- Returns up to 2 most relevant results
- Use this FIRST to identify which documents to search
2. process_query(text: str, filename: str)
- Searches within a specific document using the filename
- Returns up to 2 most relevant passages
- Use this AFTER identifying the relevant document

Remember to:
- ALWAYS ALWAYS DO NOT MAKE ANY ASSUMPTION or GENERATE SYNTHTIC DATA. IF YOU DON'T HAVE THE ANSWER, JUST SAY SO.
- Always start with query decomposition, even for seemingly simple queries.
You will provide users with well-reasoned, accurate, and comprehensive responses while demonstrating your thought process throughout the interaction.
"""

In [9]:
def chat (user_message: str):

    print(f"User: {user_message}")
    messages = []
    messages.append({"role": "user", "content": [{"text": user_message}]})
    
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        system=[{
            'text': system_message,
        }],
        toolConfig={
            'tools': tools,
            'toolChoice': {'any': {}}
        },
        inferenceConfig={
            'maxTokens': 1000,
            'temperature': 0
        }
    )

    # print(f"\nInitial Response:")
    # print(f"Stop Reason: {response['stopReason']}")
    # print(f"Content: {response['output']['message']['content']}")

    while response['stopReason'] == "tool_use":
        tool_use = next(block for block in response['output']['message']['content'] if isinstance(block, dict) and 'toolUse' in block)
        tool_name = tool_use['toolUse']['name']
        tool_input = tool_use['toolUse']['input']
        tool_use_id = tool_use['toolUse']['toolUseId']

        # print(f"\nTool Used: {tool_name}")
        # print(f"Tool Input:")
        # print(json.dumps(tool_input, indent=2))

        tool_result = process_tool_call(tool_name, tool_input)

        # print(f"\nTool Result:")
        # print(json.dumps(tool_result, indent=2))

        tool_messages = [
            {
                "role": "assistant", 
                "content": response['output']['message']['content']
            },
            {
                "role": "user",
                "content": [
                    {
                        "toolResult": {
                            "toolUseId": tool_use_id, 
                            "content": [
                                {
                                    "json": {
                                        "result": tool_result
                                    }
                                }
                            ]
                        }
                    }
                ],
            },
        ]

        messages.extend(tool_messages)

        response = bedrock_client.converse(
            modelId=model_id,
            inferenceConfig={
                'maxTokens': 1000,
                'temperature': 0,
            },
            messages=messages,
            system=[
                {
                    'text': system_message
                },
            ],
            toolConfig={"tools": tools}
        )

        # print(f"\nResponse:")
        # print(f"Stop Reason: {response['stopReason']}")
        # print(f"Content: {response['output']['message']['content']}")

    final_response = next(
        (block['text'] for block in response['output']['message']['content'] if 'text' in block),
        None,
    )

    if not final_response:
        final_response = None

    messages.append({"role": "assistant", "content": [{"text": final_response}]})
    # print(messages)

    return final_response



In [10]:
question = "What key personal event regarding Charlie Munger was mentioned in Buffett's letter to shareholders?"
final_response = chat(question)
print(f"\n{'='*50}\nFinal Response: {final_response}\n{'='*50}")

User: What key personal event regarding Charlie Munger was mentioned in Buffett's letter to shareholders?
Here is the prepared metadata filters:
{'equals': {'key': 'filename', 'value': '2023-letters.pdf'}}

Final Response: The key personal event regarding Charlie Munger mentioned in Buffett's letter to shareholders is that Charlie Munger passed away on November 28, just 33 days before his 100th birthday. Munger is credited as the architect of Berkshire Hathaway, providing crucial advice and guidance to Buffett over the years. The letter discusses Munger's influence on Buffett's investment strategies and the development of Berkshire Hathaway into a great company.
